In [1]:
import os
from IPython.display import display, Markdown
import gradio
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
load_dotenv()
key=os.getenv("DEEPSEEK_API_KEY")
if not key:
    raise ValueError("DEEPSEEK_API_KEY environment variable not set.")

In [3]:
client = OpenAI(api_key=key, base_url="https://api.deepseek.com")


In [4]:
SYSTEM_MESSAGE = "You are a helpful and friendly AI assistant. You provide concise and accurate answers."


In [5]:
def predict_ai(message, history):
    messages=[{"role":"system" , "content":SYSTEM_MESSAGE}] #messages are dict's in a list 
    for human, ai in history:
        messages.append({"role":"user","content":human})
        messages.append({"role":"assistant","content":ai})
    messages.append({"role": "user", "content": message}) # Add the current user message
    #making stream api calls to client 
    stream=client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        stream=True,
        temperature=0.8,
        max_tokens=500
    )
    response_content = ""
    # Iterate over the streamed chunks and yield them
    for chunk in stream:
        delta = chunk.choices[0].delta.content or ""
        response_content += delta
        yield response_content # Yield the accumulating response

In [19]:
chatbot=gradio.ChatInterface(
    fn=predict_ai,
    title="Simple AI Assistant",
    description="Ask me anything!",
    theme="soft",
    type="messages",
    
)

In [20]:
if __name__=="__main__":
    chatbot.launch()
    

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
